In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
project_dir = '/home/jovyan/work/MED_Fall'
work_dir = '/home/jovyan/work'

In [5]:
import os
os.environ['PYTHONPATH']

'/Users/andrea/Documents/Github/MED_Fall'

In [6]:
import sys
for p in sys.path:
    print(p)

/Applications/DataSpell.app/Contents/plugins/python-ce/helpers-pro/jupyter_debug
/Applications/DataSpell.app/Contents/plugins/python-ce/helpers/pydev
/Users/andrea/Documents/Github/MED_Fall/vision/pre_processing
/Users/andrea/Documents/Github/MED_Fall
/Users/andrea/miniforge3/envs/ml/lib/python39.zip
/Users/andrea/miniforge3/envs/ml/lib/python3.9
/Users/andrea/miniforge3/envs/ml/lib/python3.9/lib-dynload

/Users/andrea/.local/lib/python3.9/site-packages
/Users/andrea/miniforge3/envs/ml/lib/python3.9/site-packages


In [8]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model

from vision.pre_processing.load_frames_from_videos import FramesExtractor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

## Testing VGG16 Fine Tuning

In [ ]:
IMG_SIZE = (224, 224, 3)

In [ ]:
feature_extractor = tf.keras.applications.vgg16.VGG16(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=IMG_SIZE,
    pooling='avg',
)

In [ ]:
feature_extractor.summary()

In [ ]:
import numpy as np

In [ ]:
sample = np.ones(shape=IMG_SIZE)
sample = np.expand_dims(sample, axis=0)

In [ ]:
sample.shape

In [ ]:
# sample=preprocess_input(sample)
# sample.shape
# sample

In [ ]:
out = feature_extractor.predict(sample)

In [ ]:
out.shape

In [ ]:
feature_extractor

In [ ]:
# Freeze four convolution blocks
for layer in feature_extractor.layers[:15]:
    layer.trainable = False
# Make sure you have frozen the correct layers
for i, layer in enumerate(feature_extractor.layers):
    print(i, layer.name, layer.trainable)

In [ ]:
x = feature_extractor.output
x = Flatten()(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(units=256, activation='relu')(x)
x = Dense(units=3, activation='softmax')(x)

transfer_model = Model(inputs=feature_extractor.input, outputs=x)

In [ ]:
transfer_model.predict(np.expand_dims(X[0], axis=0))

In [ ]:
X = np.ones(shape=(1000, 224, 224, 3))
X.shape

In [ ]:
y = [1] * 300 + [2] * 300 + [3] * 400
y = np.array(y)
y.shape

In [ ]:
transfer_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics='acc')

In [ ]:
transfer_model.fit(epochs=10, x=X, y=y)

In [ ]:
last = len(transfer_model.layers)
last

In [ ]:
type(transfer_model.layers)

In [ ]:
#transfer_model.layers[-5:]
from tensorflow.keras.models import Sequential

feature_extractor_tuned = Sequential(transfer_model.layers[:-5])

In [ ]:
feature_extractor_tuned.summary()

In [ ]:
pred = feature_extractor_tuned.predict(np.expand_dims(X[0], axis=0))
pred.shape

## Actual fine tuning pipeline

In [16]:
fe = FramesExtractor(videos_folder=f"../../dataset/data/test_videos", output_folder='/Users/andrea/Desktop/extracted_frames', labels="../../dataset/labels.xlsx",
                     ground_truth_folder="../vision_dataset/ground_truth")
extracted_frames, ground_truth = fe.extract_frames()
print('one frame shape: ', extracted_frames[0].shape)
print('extracted_frames shape: ', extracted_frames.shape)
print('ground truth shape: ', ground_truth.shape)

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/4920 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
extracted_frames.shape

In [ ]:
ground_truth.head()

### Train Test Split

In [ ]:
X = extracted_frames
y = ground_truth['macro_labels']

enc = LabelEncoder()
y = enc.fit_transform(y)

print(enc.categories_)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=2, shuffle=True)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

### Build the model to fine tune

In [9]:
IMG_SIZE = (224,224,3)

In [10]:
feature_extractor = tf.keras.applications.vgg16.VGG16(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=IMG_SIZE,
    pooling='avg',
)

feature_extractor.summary()


2022-07-24 10:44:36.781373: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-24 10:44:36.781425: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: 6d2e41cc007a
2022-07-24 10:44:36.781436: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: 6d2e41cc007a
2022-07-24 10:44:36.781604: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.47.3
2022-07-24 10:44:36.781627: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.129.6
2022-07-24 10:44:36.781636: E tensorflow/stream_executor/cuda/cuda_diagnostics.cc:313] kernel version 470.129.6 does not match DSO version 510.47.3 -- cannot find working devices in this configuration
2022-07-24 10:44:36.782168: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI

58889256/58889256 [==============================] - 5s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [11]:
#freeze all layers except last five
for layer in feature_extractor.layers[:-5]:
    layer.trainable = False

for i, layer in enumerate(feature_extractor.layers):
    print(i, layer.name, layer.trainable)

0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 True
16 block5_conv2 True
17 block5_conv3 True
18 block5_pool True
19 global_average_pooling2d True


In [12]:
x = feature_extractor.output
x = Flatten()(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(units=256, activation='relu')(x)
x = Dense(units=3, activation='softmax')(x)

transfer_model = Model(inputs=feature_extractor.input, outputs=x)

transfer_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [13]:
transfer_model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam')

In [ ]:
history = transfer_model.fit(x=X, y=y, validation_data = (X_test, y_test), epochs=10, metrics='accuracy')

In [ ]:
##plot metrics